In [1]:
import datetime as dt
import pandas as pd
import warnings
import pymssql
import locale

warnings.filterwarnings('ignore')
locale.setlocale(locale.LC_ALL, 'ru_RU.UTF-8')

'ru_RU.UTF-8'

In [2]:
conn = pymssql.connect(server='spb-s-sql-dwh', database='DWH_DEV', charset='WINDOWS-1251')
cursor = conn.cursor()

productimage = f'''
select 
[image], [sort], [product_id], [deleted]
from [DWH_DEV].[ecom].[catalog_productimage]
where [deleted] is NULL
'''

product = f'''
select 
[id], [article], [published]
from [DWH_DEV].[ecom].[catalog_product]
'''

catalog_productimage = pd.read_sql_query(productimage, conn)
catalog_product = pd.read_sql_query(product, conn)

In [3]:
# catalog_product = catalog_product.query('published == True')
catalog_productimage.rename(columns={'product_id': 'id'},
                            inplace=True)
photos = catalog_product.merge(catalog_productimage, how='left', on='id')
photos['image'] = 'https://static2.585.cloud/media/' + photos['image']
photos = photos.query('sort == 1')
photos = photos[['article', 'published', 'image']]
photos.rename(columns={'article': 'ID Сайта', 'image': 'Ссылка на изображение'}, inplace=True)
date = dt.date.today().strftime('%d.%m.%Y')
photos.to_excel(fr'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\Тренкин Сергей\Фото с сайта для отчётности\фото с сайта {date}.xlsx', index=False)

In [12]:
filename = r'C:\Users\Trenkin.Sergey\Desktop\Остаток Коламбус по 3 ступени_1.xlsx'
df = pd.read_excel(filename)
df['ID Сайта'] = df['ID Сайта'].astype(str)

In [15]:
df = df.merge(photos, how='left', on='ID Сайта')

In [16]:
df.to_excel(filename, index=False)